### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221203_scenario_projections.ipynb")

2022/12/02 08:43:42 INFO|============================================================


### Fit an initial scenario through present day

In [3]:
# set inputs for the model

# first establish the model start date
startdate = dt.date(2020, 1, 24).isoformat()
# then get today's date (so we can set end dates automatically)
today = dt.datetime.today().strftime("%Y-%m-%d")
# now calculate the end date (let's do 90 days from today's date, but can be as long we we want)
plus90 = dt.datetime.strptime(today, "%Y-%m-%d").date() + dt.timedelta(days = 90)
enddate = plus90.isoformat()

# set model arguments
model_args = {
    'params_defs': 'covid_model/analysis/20221202_dec_cdphe_slides/20221202_scenario_params.json',
    #'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221202_dec_cdphe_slides/20221202_vacc_proj_params.json',
    'start_date': startdate,
    'end_date': enddate
}

# set output directory and end date for fit (which is today's date)
fit_args = {
    'outdir': outdir,
    'fit_end_date': '2022-12-01'
}

In [4]:
# if you want to refit the model, run this code
# this creates an object called "model" by passing the fit_args and model_args through the do_single_fit function
# otherwise, run the next cell down
model = do_single_fit(**fit_args, **model_args)



2022/12/02 08:43:49 INFO|-------------------------|{} Retrieving vaccinations data
2022/12/02 08:43:51 INFO|-------------------------|{} Constructing vaccination projections
2022/12/02 08:43:52 INFO|-------------------------|{} Retrieving hospitalizations data
2022/12/02 08:43:54 INFO|----------------------|{} Prepping Model
2022/12/02 08:43:54 INFO|-----------------------|{} Prepping Model
2022/12/02 08:45:09 INFO|----------------------|{} Model prepped for fitting in 75.62295375 seconds.
2022/12/02 08:45:09 INFO|----------------------|{} Will fit 35 times
2022/12/02 08:46:45 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/35 completed in 96.230994917 seconds: {0: {'co': 0.8306366901049832}, 14: {'co': 0.6979474721369532}, 28: {'co': 0.5697936096821077}, 42: {'co': 0.0028740584700378217}, 56: {'co': 0.6814675459889091}, 70: {'co': 0.8912767480731285}}
2022/12/02 08:46:45 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/12/02 08:47:06

2022/12/02 08:52:07 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/35 completed in 9.870570416999954 seconds: {504: {'co': 0.7927614871209054}, 518: {'co': 0.7862978554746909}, 532: {'co': 0.7661786113032724}, 546: {'co': 0.7176009765946489}, 560: {'co': 0.7506313141027119}, 574: {'co': 0.7457868867473114}}
2022/12/02 08:52:07 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/12/02 08:52:26 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/35 completed in 17.328650249999953 seconds: {532: {'co': 0.7655333902579111}, 546: {'co': 0.7194148609709348}, 560: {'co': 0.7472635843220005}, 574: {'co': 0.7558547271989949}, 588: {'co': 0.7926874525758997}, 602: {'co': 0.7730462154883716}}
2022/12/02 08:52:26 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/12/02 08:52:43 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/35 completed in 15.135060708999958 seconds: 

2022/12/02 08:58:13 INFO|-----------------------|{'fit_batch': '34', 'run_type': 'fit'}: Running forward sim
2022/12/02 08:58:14 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'}: Uploading final results
2022/12/02 08:58:22 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'}: spec_id: 4519


In [ ]:
# if you want to pass an old fit spec ID through, run this code
# otherwise, refit using the code above
#model_args = {
    #'base_spec_id': 4515, # use the spec id that was output from the model fit you want to pass through
#}
# create an object called "model" by passing the above-mentioned model_args through CovidModel
#model = CovidModel(**model_args)
# prep the model
#model.prep()

### Run the model

In [6]:
# now create the reports for the model, indexing through all the values of TC we want to simulate
logging.info('Projecting')
for tc in [0.7961, 0.8153, 0.8428]:
    logging.info('')
    #model.update_tc({model.date_to_t("2022-11-29"): {'co': tc}}, replace = False)
    model.update_tc({model.date_to_t(today): {'co': tc}}, replace = False)
    model.tags.update({'tc': tc})
    model.solve_seir()
    do_create_report(model, outdir=outdir, prep_model=False, solve_model=False, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), enddate)
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), enddate)
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

    logging.info('Running reports')

2022/12/02 09:08:28 INFO|Projecting
2022/12/02 09:08:28 INFO|
2022/12/02 09:08:35 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/02 09:10:31 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.7961}: Running forward sim
2022/12/02 09:10:32 INFO|Running reports
2022/12/02 09:10:32 INFO|
2022/12/02 09:10:39 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/02 09:12:38 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.8153}: Running forward sim
2022/12/02 09:12:39 INFO|Running reports
2022/12/02 09:12:39 INFO|
2022/12/02 09:12:48 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/02 09:14:46 INFO|{'fit_batch': '34', 'run_t